## 📦 Setup

In [ ]:
# 1. Carregar dados + 
import sys
sys.path.append('..')  # Adiciona o diretório pai
from src.utils.io.io_local import *
from src.utils.io.io_clearml import *

from src.utils.io import load_dataframe
from config import config_custom as config
from src.pipelines.pipeline_processamento import executar_pipeline_processamento
from src.pipelines.pipeline_features import executar_pipeline_features
from src.pipelines import treinar_pipeline_completo, treinar_rapido

df_raw = load_dataframe('../dados/2025.05.14_thermal_confort_santa_maria_brazil_.csv')
print(f'Dados brutos: {df_raw.shape}')

# 1. Pipelines (local/standalone)

## 1.1 Pipeline processamento

In [ ]:
# 1. Carregar dados + # 2. PROCESSAMENTO (Limpeza + Imputação)
df_raw = load_dataframe('../dados/2025.05.14_thermal_confort_santa_maria_brazil_.csv')
print(f'Dados brutos: {df_raw.shape}')

# 2. PROCESSAMENTO (Limpeza + Imputação)
df_proc = executar_pipeline_processamento(
    df_raw,
    config_imputacao_customizada=config.CONFIG_IMPUTACAO_CUSTOMIZADA,
    criar_agrupamento_temporal=True,
    nome_coluna_agrupamento='mes-ano'
)
print(f'Após processamento: {df_proc.shape}')
print(f'NAs restantes: {df_proc.isna().sum().sum()}')

## 1.2 Pipeline features

In [ ]:
# 3. FEATURES (Codificação + Derivadas + Normalização)
df_feat, artefatos = executar_pipeline_features(
    df_proc,
    # Codificação
    aplicar_codificacao=True,
    metodo_codificacao='label',           # 'label' ou 'onehot'
    sufixo_codificacao='_cod',
    
    # Features derivadas
    criar_features_derivadas=True,
    tipos_features_derivadas=[
        'imc',                            # Índice de Massa Corporal
        'imc_classe',                     # Classe do IMC
        'heat_index',                     # Índice de calor
        'dew_point',                      # Ponto de orvalho
        't*u',                            # Temperatura × Umidade
    ],
    
    # Normalização
    aplicar_normalizacao=True,
    metodo_normalizacao='standard',       # 'standard', 'minmax', 'robust'
    agrupamento_normalizacao='mes-ano',   # Normalizar por grupo
    sufixo_normalizacao='_norm',
)

print(f'Após features: {df_feat.shape}')
print(f'Artefatos criados: {list(artefatos.keys())}')
df_feat.to_csv("../dados/resultados/dados_processados_novas_features.csv")
# Visualizar resultado
df_feat.head()


## 1.3 Pipeline de treinamento

In [ ]:
from config.config_gerais import PARAMS_PADRAO

# Definir coluna alvo e features
coluna_alvo = 'p1'

# Usar apenas features principais para teste
features_treino = [
    'idade', 'sexo_cod', 'peso', 'altura',
    'tmedia', 'ur', 'vel_vento',
]

tipos_modelos ='regressao'

# Filtrar features que existem no DataFrame
features_existentes = [f for f in features_treino if f in df_feat.columns]
print(f"Features para treinamento: {features_existentes}")

# Preparar dados
df_treino = df_feat[features_existentes + [coluna_alvo]].dropna()
print(f"Dataset de treino: {df_treino.shape}")

# 4. TREINAMENTO
print("\n" + "="*60)
print("🤖 INICIANDO PIPELINE DE TREINAMENTO")
print("="*60)

resultado = treinar_pipeline_completo(
    dados=df_treino,
    coluna_alvo=coluna_alvo,
    tipo_problema=tipos_modelos,  # 'classificacao' ou 'regressao'
    params_setup=PARAMS_PADRAO,
    n_modelos_comparar=3,           # Testar top 3 modelos
    otimizar_hiperparametros=True,         # Otimizar hiperparâmetros
    n_iter_otimizacao=10,  # 10 iterações de otimização
    salvar_modelo_final=True,           # Salvar modelo
    nome_modelo="modelo_conforto_termico",
    pasta_modelos="modelos"
)

# Visualizar resultados
print("\n" + "="*60)
print("📊 RESULTADOS DO TREINAMENTO")
print("="*60)

# Nome do melhor modelo
nome_modelo = str(resultado['tabela_comparacao'].index[0])
print(f"\n✓ Melhor modelo: {nome_modelo}")

print(f"\n📈 Métricas principais:")
metricas = resultado['metricas_melhor']
for nome, valor in metricas.items():
    if isinstance(valor, (int, float)):
        print(f"  • {nome}: {valor:.4f}")

print(f"\n💾 Modelo salvo em: {resultado.get('caminho_modelo', 'N/A')}")

print("\n📋 Comparação de modelos (top 5 métricas):")
print(resultado['tabela_comparacao'].head())

In [ ]:
resultados_10_experimentos = {}
for i in range(10):
    print("\n" + "="*60)
    print(f"🤖 INICIANDO EXPERIMENTO {i+1}/10")
    print("="*60)
    
    resultado = treinar_pipeline_completo(
    dados=df_treino,
    coluna_alvo=coluna_alvo,
    tipo_problema='regressao',  # 'classificacao' ou 'regressao'
    params_setup=PARAMS_PADRAO,
    n_modelos_comparar=3,           # Testar top 3 modelos
    otimizar_hiperparametros=True,         # Otimizar hiperparâmetros
    n_iter_otimizacao=10,  # 10 iterações de otimização
    salvar_modelo_final=True,           # Salvar modelo
    nome_modelo="modelo_conforto_termico",
    pasta_modelos="modelos"
)
    
    resultados_10_experimentos[f'Experimento_{i+1}'] = resultado


In [ ]:
tabelas_comparacao = [dicionario["tabela_comparacao"] for dicionario in resultados_10_experimentos.values()] 
tabelas_comparacao =  [tabela.rename(columns={'Model':'Modelos','Accuracy': 'Acurácia', 'AUC': 'AUC', 'Recall': 'Recall', 'Prec.': 'Prec.', 'F1': 'F1'}) for tabela in tabelas_comparacao]
serie_nomes_modelos = tabelas_comparacao[0]['Modelos']
tabelas_comparacao = [tabela.select_dtypes(include='number') for tabela in tabelas_comparacao] 

In [ ]:

df_desvio_metricas= pd.concat(tabelas_comparacao).groupby(level=0).std()
df_media_metricas = pd.concat(tabelas_comparacao).groupby(level=0).mean()
df_media_desvio_metricas_str = df_media_metricas.round(2).astype(str) + " ± " + df_desvio_metricas.round(2).astype(str)
df_media_desvio_metricas_str = df_media_desvio_metricas_str.merge(serie_nomes_modelos, left_index=True, right_index=True)
df_media_metricas = df_media_metricas.merge(serie_nomes_modelos, left_index=True, right_index=True)
df_desvio_metricas = df_desvio_metricas.merge(serie_nomes_modelos, left_index=True, right_index=True)
df_desvio_metricas.to_csv('../dados/resultados/desvio_metricas.csv')
df_media_metricas.to_csv('../dados/resultados/media_metricas.csv')
df_media_desvio_metricas_str.to_csv('../dados/resultados/media_e_desvio_metricas_str.csv')

In [ ]:
df_media_desvio_metricas_str

# Pipelines com integração ClearML

In [ ]:
# =========================================================================
# PIPELINE COMPLETO COM CLEARML - REGISTRO TOTAL
# =========================================================================
import sys
sys.path.append('..')

from clearml import Task, Dataset, OutputModel
import pandas as pd
from pathlib import Path

# Imports dos pipelines
from src.pipelines.pipeline_processamento import executar_pipeline_processamento
from src.pipelines.pipeline_treinamento_unified import treinar_pipeline_completo

# Configuração
PROJECT_NAME = "conforto_termico"
coluna_alvo = 'p1'

print("="*80)
print("PIPELINE COMPLETO COM CLEARML - FULL TRACKING")
print("="*80)

# ============================================================================
# ETAPA 1: UPLOAD DE DADOS BRUTOS
# ============================================================================
print("\n[ETAPA 1] Upload de Dados Brutos para ClearML")
print("-"*80)

# Criar dataset de dados brutos
dataset_bruto = Dataset.create(
    dataset_name="dados_brutos_conforto_termico",
    dataset_project=PROJECT_NAME,
    description="Dados brutos de conforto termico de Santa Maria"
)

# Salvar temporariamente para upload
temp_path = Path("../dados/temp_clearml")
temp_path.mkdir(exist_ok=True)
arquivo_bruto = temp_path / "dados_brutos.csv"
df_raw.to_csv(arquivo_bruto, index=False)

# Adicionar arquivo ao dataset
dataset_bruto.add_files(str(arquivo_bruto))
dataset_bruto.upload()
dataset_bruto.finalize()

print(f"Dataset bruto criado: ID = {dataset_bruto.id}")
print(f"  - Shape: {df_raw.shape}")
print(f"  - Colunas: {len(df_raw.columns)}")

# ============================================================================
# ETAPA 2: PIPELINE DE PROCESSAMENTO
# ============================================================================
print("\n[ETAPA 2] Pipeline de Processamento")
print("-"*80)

# Criar task de processamento (COM FIX PARA NOTEBOOKS)
task_processamento = Task.init(
    project_name=PROJECT_NAME,
    task_name="Pipeline_Processamento",
    task_type=Task.TaskTypes.data_processing,
    reuse_last_task_id=False,  # Não reusar task anterior
    continue_last_task=False,   # Criar nova task sempre
    auto_connect_frameworks=False,  # Desabilita captura automática
    auto_resource_monitoring=False  # Desabilita monitoramento de recursos
)

# Conectar dataset de entrada
task_processamento.connect_configuration({
    "dataset_id": dataset_bruto.id,
    "dataset_name": "dados_brutos_conforto_termico"
})

# Executar processamento
print("Executando processamento...")
df_processado = executar_pipeline_processamento(df_raw)

# Registrar métricas de processamento
task_processamento.get_logger().report_single_value("linhas_entrada", df_raw.shape[0])
task_processamento.get_logger().report_single_value("linhas_saida", df_processado.shape[0])
task_processamento.get_logger().report_single_value("colunas_entrada", df_raw.shape[1])
task_processamento.get_logger().report_single_value("colunas_saida", df_processado.shape[1])
task_processamento.get_logger().report_single_value("nas_removidos", df_raw.isna().sum().sum() - df_processado.isna().sum().sum())

print(f"Dados processados: {df_processado.shape}")
print(f"NAs restantes: {df_processado.isna().sum().sum()}")

# Upload de dados processados como dataset
dataset_processado = Dataset.create(
    dataset_name="dados_processados_conforto_termico",
    dataset_project=PROJECT_NAME,
    parent_datasets=[dataset_bruto.id],
    description="Dados processados (sem NAs, limpeza aplicada)"
)

arquivo_processado = temp_path / "dados_processados.csv"
df_processado.to_csv(arquivo_processado, index=False)
dataset_processado.add_files(str(arquivo_processado))
dataset_processado.upload()
dataset_processado.finalize()

print(f"Dataset processado criado: ID = {dataset_processado.id}")

# Registrar como artefato na task
task_processamento.upload_artifact(
    "dados_processados_sample",
    artifact_object=df_processado.head(100)  # Apenas amostra
)

task_processamento.close()
print("Task de processamento finalizada!")

# ============================================================================
# ETAPA 3: PIPELINE DE TREINAMENTO
# ============================================================================
print("\n[ETAPA 3] Pipeline de Treinamento")
print("-"*80)

# Criar task de treinamento (COM FIX PARA NOTEBOOKS)
task_treinamento = Task.init(
    project_name=PROJECT_NAME,
    task_name="Pipeline_Treinamento",
    task_type=Task.TaskTypes.training,
    reuse_last_task_id=False,
    continue_last_task=False,
    auto_connect_frameworks=False,
    auto_resource_monitoring=False
)

# Conectar dataset processado
task_treinamento.connect_configuration({
    "dataset_id": dataset_processado.id,
    "coluna_alvo": coluna_alvo,
    "tipo_problema": "regressao"
})

# Preparar dados
df_treino = df_processado.dropna()
print(f"Dados para treino: {df_treino.shape}")

# Executar treinamento
print("Executando treinamento...")
resultado = treinar_pipeline_completo(
    dados=df_treino,
    coluna_alvo=coluna_alvo,
    tipo_problema='regressao',
    n_modelos_comparar=3,
    otimizar_hiperparametros=True,
    n_iter_otimizacao=10,
    salvar_modelo_final=True,
    nome_modelo="modelo_conforto_termico_clearml",
    pasta_modelos="../modelos"
)

# ============================================================================
# ETAPA 4: REGISTRO DE MÉTRICAS
# ============================================================================
print("\n[ETAPA 4] Registro de Métricas")
print("-"*80)

logger = task_treinamento.get_logger()

# Métricas do melhor modelo
metricas = resultado['metricas_melhor']
print("\nMetricas registradas:")
for nome, valor in metricas.items():
    if isinstance(valor, (int, float)):
        logger.report_single_value(nome, valor)
        print(f"  {nome}: {valor:.4f}")

# ============================================================================
# ETAPA 5: UPLOAD DE TABELAS E ARTEFATOS
# ============================================================================
print("\n[ETAPA 5] Upload de Tabelas e Artefatos")
print("-"*80)

# Tabela de comparação de modelos
tabela_comparacao = resultado['tabela_comparacao']
task_treinamento.upload_artifact(
    "comparacao_modelos",
    artifact_object=tabela_comparacao
)
print("Tabela de comparacao enviada como artefato")

# Criar gráfico de comparação de modelos
import matplotlib.pyplot as plt

fig, ax = plt.subplots(figsize=(10, 6))
tabela_comparacao[['MAE', 'MSE', 'RMSE', 'R2']].plot(kind='bar', ax=ax)
ax.set_title('Comparacao de Modelos - Metricas')
ax.set_xlabel('Modelos')
ax.set_ylabel('Valor')
plt.xticks(rotation=45, ha='right')
plt.tight_layout()

# Enviar gráfico
logger.report_matplotlib_figure(
    title="Comparacao de Modelos",
    series="Metricas",
    figure=fig,
    iteration=0
)
plt.close()
print("Grafico de comparacao enviado")

# ============================================================================
# ETAPA 6: REGISTRO DO MODELO
# ============================================================================
print("\n[ETAPA 6] Registro do Modelo")
print("-"*80)

# Criar OutputModel
melhor_modelo_nome = str(resultado['tabela_comparacao'].index[0])
output_model = OutputModel(
    task=task_treinamento,
    name=f"modelo_{melhor_modelo_nome}_conforto_termico",
    framework="PyCaret"
)

# Registrar informações do modelo
output_model.update_labels({
    "tipo": "regressao",
    "coluna_alvo": coluna_alvo,
    "melhor_modelo": melhor_modelo_nome
})

# Adicionar métricas ao modelo
output_model.update_design(config_dict={
    "metricas": {k: float(v) for k, v in metricas.items() if isinstance(v, (int, float))},
    "n_features": df_treino.shape[1] - 1,
    "n_samples_treino": df_treino.shape[0]
})

# Upload do arquivo do modelo
if 'caminho_modelo' in resultado and resultado['caminho_modelo']:
    caminho_modelo = Path(resultado['caminho_modelo'])
    if caminho_modelo.exists():
        output_model.update_weights(weights_filename=str(caminho_modelo))
        print(f"Modelo registrado: {caminho_modelo}")
    else:
        print(f"AVISO: Arquivo do modelo nao encontrado: {caminho_modelo}")

# ============================================================================
# ETAPA 7: FINALIZAÇÃO E RESUMO
# ============================================================================
print("\n[ETAPA 7] Finalizacao")
print("-"*80)

# Tags na task
task_treinamento.add_tags([
    "producao",
    melhor_modelo_nome,
    f"r2_{metricas.get('R2', 0):.2f}".replace(".", "_")
])

task_treinamento.close()

# Limpar arquivos temporários
import shutil
if temp_path.exists():
    shutil.rmtree(temp_path)

# ============================================================================
# RESUMO FINAL
# ============================================================================
print("\n" + "="*80)
print("PIPELINE COMPLETO FINALIZADO - TUDO REGISTRADO NO CLEARML!")
print("="*80)
print(f"\nProjeto: {PROJECT_NAME}")
print(f"\nDatasets criados:")
print(f"  1. Dados brutos: {dataset_bruto.id}")
print(f"  2. Dados processados: {dataset_processado.id}")
print(f"\nTasks criadas:")
print(f"  1. Processamento: {task_processamento.id}")
print(f"  2. Treinamento: {task_treinamento.id}")
print(f"\nModelo registrado:")
print(f"  - Nome: modelo_{melhor_modelo_nome}_conforto_termico")
print(f"  - Melhor modelo: {melhor_modelo_nome}")
print(f"  - R2: {metricas.get('R2', 0):.4f}")
print(f"  - MAE: {metricas.get('MAE', 0):.4f}")
print("\nAcesse o ClearML UI para visualizar todos os resultados!")
print("="*80)

## [OPCIONAL] Buscar e Reusar Recursos do ClearML

In [ ]:
# =========================================================================
# REUSAR RECURSOS DO CLEARML (Datasets, Modelos, Tasks)
# =========================================================================
import sys
sys.path.append('..')

from clearml import Task, Dataset, Model
import pandas as pd

PROJECT_NAME = "conforto_termico_def"

print("="*80)
print("BUSCANDO RECURSOS NO CLEARML")
print("="*80)

# ============================================================================
# 1. LISTAR DATASETS DISPONÍVEIS
# ============================================================================
print("\n[1] Datasets disponiveis no projeto:")
print("-"*80)

datasets = Dataset.list_datasets(
    dataset_project=PROJECT_NAME,
    partial_name="conforto_termico"
)

for i, ds in enumerate(datasets, 1):
    print(f"\n{i}. {ds.name}")
    print(f"   ID: {ds.id}")
    print(f"   Versao: {ds.version}")
    print(f"   Criado: {ds.created}")

# ============================================================================
# 2. BAIXAR DATASET ESPECÍFICO
# ============================================================================
print("\n[2] Baixando dataset processado:")
print("-"*80)

# Buscar o dataset mais recente de dados processados
dataset_processado = Dataset.get(
    dataset_project=PROJECT_NAME,
    dataset_name="dados_processados_conforto_termico"
)

# Baixar para pasta local
local_path = dataset_processado.get_local_copy()
print(f"Dataset baixado para: {local_path}")

# Carregar dados
arquivos = list(Path(local_path).glob("*.csv"))
if arquivos:
    df_downloaded = pd.read_csv(arquivos[0])
    print(f"Dados carregados: {df_downloaded.shape}")
    print(df_downloaded.head())

# ============================================================================
# 3. LISTAR TASKS DO PROJETO
# ============================================================================
print("\n[3] Tasks do projeto:")
print("-"*80)

tasks = Task.get_tasks(
    project_name=PROJECT_NAME,
    task_name=None  # Todas as tasks
)

for i, t in enumerate(tasks[:5], 1):  # Primeiras 5
    print(f"\n{i}. {t.name}")
    print(f"   ID: {t.id}")
    print(f"   Status: {t.status}")
    print(f"   Tipo: {t.task_type}")

# ============================================================================
# 4. BUSCAR MODELOS REGISTRADOS
# ============================================================================
print("\n[4] Modelos registrados:")
print("-"*80)

modelos = Model.query_models(
    project_name=PROJECT_NAME,
    model_name=None  # Todos os modelos
)

for i, modelo in enumerate(modelos, 1):
    print(f"\n{i}. {modelo.name}")
    print(f"   ID: {modelo.id}")
    print(f"   Framework: {modelo.framework}")
    print(f"   Labels: {modelo.labels}")
    
    # Baixar modelo se quiser
    # modelo_local = modelo.get_local_copy()
    # print(f"   Download: {modelo_local}")

# ============================================================================
# 5. CLONAR E REUSAR TASK
# ============================================================================
print("\n[5] Exemplo: Clonar uma task existente")
print("-"*80)

if tasks:
    task_original = tasks[0]
    print(f"Clonando task: {task_original.name}")
    
    # Clonar task (cria uma cópia)
    task_clonada = Task.clone(
        source_task=task_original.id,
        name=f"{task_original.name}_clonada",
        project=PROJECT_NAME
    )
    
    print(f"Task clonada criada: {task_clonada.id}")
    print("Você pode executar esta task clonada com configuracoes diferentes!")

print("\n" + "="*80)
print("BUSCA DE RECURSOS CONCLUIDA")
print("="*80)

In [1]:
# PASSO 1: Verificar Importações e Credenciais
# ============================================================================

import sys
from pathlib import Path

# Adicionar path do projeto
if '..' not in sys.path:
    sys.path.append('..')

from dotenv import load_dotenv
load_dotenv()

# 2. Testar carregamento de credenciais
print("\n[2] Testando carregamento de credenciais...")
try:
    from src.clearml.utils.credenciais_clearml import configurar_clearml_online
    sucesso = configurar_clearml_online()
    if sucesso:
        print("✓ Credenciais carregadas com sucesso")
    else:
        print("✗ Falha ao carregar credenciais")
except Exception as e:
    print(f"✗ Erro ao importar/configurar: {e}")
    import traceback
    traceback.print_exc()

# 3. Verificar importação do pipeline
print("\n[3] Testando importação do pipeline...")
try:
    from src.clearml.pipelines_clearml import executar_pipeline_processamento_clearml
    print("✓ Pipeline importado com sucesso")
except Exception as e:
    print(f"✗ Erro ao importar pipeline: {e}")
    import traceback
    traceback.print_exc()

# 4. Verificar arquivo de dados
print("\n[4] Verificando arquivo de dados...")
arquivo_dados = Path('../dados/2025.05.14_thermal_confort_santa_maria_brazil_.csv')
if arquivo_dados.exists():
    print(f"✓ Arquivo encontrado: {arquivo_dados.resolve()}")
    print(f"  Tamanho: {arquivo_dados.stat().st_size / (1024*1024):.2f} MB")
else:
    print(f"✗ Arquivo NÃO encontrado: {arquivo_dados.resolve()}")

print("\n" + "="*80)
print("Se tudo estiver ✓, execute a próxima célula para rodar o pipeline")
print("="*80)


[2] Testando carregamento de credenciais...


2026-02-10 12:31:24,996 - INFO - Módulo ClearML carregado com sucesso
2026-02-10 12:31:24,999 - INFO - ✓ Credenciais ClearML carregadas do .env
2026-02-10 12:31:25,000 - INFO -   API Host: https://api.clear.ml
2026-02-10 12:31:25,001 - INFO -   Web Host: https://app.clear.ml
2026-02-10 12:31:25,001 - INFO - ✓ ClearML configurado para uso ONLINE


✓ Credenciais carregadas com sucesso

[3] Testando importação do pipeline...
✓ Pipeline importado com sucesso

[4] Verificando arquivo de dados...
✓ Arquivo encontrado: C:\Arquivos_Pessoais\PROJETOS\tcc_mba_esalq_mlops\dados\2025.05.14_thermal_confort_santa_maria_brazil_.csv
  Tamanho: 0.29 MB

Se tudo estiver ✓, execute a próxima célula para rodar o pipeline


In [2]:
from dotenv import load_dotenv
load_dotenv()

from clearml import Task

# O ClearML vai usar as variáveis de ambiente automaticamente
task = Task.init(project_name='teste', task_name='meu_teste')
task.close()

ClearML Task: overwriting (reusing) task id=33e72bc5f5b14b8fae2b2335b2e97e15
ClearML results page: https://app.clear.ml/projects/ff08d1c0eb5e473885441836f34c7e39/experiments/33e72bc5f5b14b8fae2b2335b2e97e15/output/log
2026-02-10 12:31:57,689 - clearml.Task - INFO - Waiting for repository detection and full package requirement analysis
2026-02-10 12:32:55,157 - clearml.Task - INFO - Finished repository detection and package analysis


In [3]:
# Processamento
# ============================================================================
import traceback
from src.clearml.pipelines_clearml import executar_pipeline_processamento_clearml

arquivo = '../dados/2025.05.14_thermal_confort_santa_maria_brazil_.csv'

resultado = executar_pipeline_processamento_clearml(
    caminho_csv=arquivo,
    offline_mode=False
)

df_features = resultado['dados_processados']



2026-02-10 12:33:04,058 - INFO - Modo ONLINE: Iniciando integração ClearML
2026-02-10 12:33:04,065 - INFO - ✓ Credenciais ClearML carregadas do .env
2026-02-10 12:33:04,065 - INFO -   API Host: https://api.clear.ml
2026-02-10 12:33:04,065 - INFO -   Web Host: https://app.clear.ml
2026-02-10 12:33:04,065 - INFO - ✓ ClearML configurado para uso ONLINE


ClearML Task: created new task id=877bf2def79d4e66b6b53d22beb00aea
ClearML results page: https://app.clear.ml/projects/06661800c2e642339d895b6d02c06481/experiments/877bf2def79d4e66b6b53d22beb00aea/output/log


2026-02-10 12:33:35,556 - INFO - Task criada: Pipeline_Processamento (ID: 877bf2def79d4e66b6b53d22beb00aea, Projeto: Estudo de Sensação Térmica Humana em Santa Maria - RS)
2026-02-10 12:34:25,843 - INFO - ✓ Task ClearML criada (ID: 877bf2def79d4e66b6b53d22beb00aea)
2026-02-10 12:34:25,844 - INFO - ================================================================================
2026-02-10 12:34:25,845 - INFO - PIPELINE DE PROCESSAMENTO
2026-02-10 12:34:25,846 - INFO - ================================================================================
2026-02-10 12:34:25,846 - INFO - 
[1] Carregando dados do CSV...
2026-02-10 12:34:25,847 - INFO -     Arquivo: ../dados/2025.05.14_thermal_confort_santa_maria_brazil_.csv
2026-02-10 12:34:25,875 - INFO -     ✓ Dados carregados: (1720, 40)
2026-02-10 12:34:25,876 - INFO -     Colunas: 40
2026-02-10 12:34:25,877 - INFO - 
[2] Executando pipeline de processamento local...
2026-02-10 12:34:25,965 - INFO - ✓ Processamento concluído
2026-02-10 12:34

🔄 Iniciando pipeline de processamento BASE...
  1️⃣ Aplicando substituições de limpeza...
  2️⃣ Convertendo tipos de dados...
  3️⃣ Imputando valores faltantes...
  4️⃣ Criando agrupamento temporal...
✅ Pipeline BASE concluído! Shape final: (1720, 41)


2026-02-10 12:34:39,125 - INFO - DataFrame registrado: dados_processados_sample (shape: (100, 41))
2026-02-10 12:34:39,126 - INFO - ✓ Artefatos registrados
2026-02-10 12:34:39,126 - INFO - 
[4] Criando dataset ClearML...


ClearML results page: https://app.clear.ml/projects/801c2956f7704234b987beef3f9cc1e6/experiments/73e2a80ce490442eb0682b2bcbd5ff00/output/log
ClearML dataset page: https://app.clear.ml/datasets/simple/801c2956f7704234b987beef3f9cc1e6/experiments/73e2a80ce490442eb0682b2bcbd5ff00


2026-02-10 12:35:56,478 - INFO - Dataset criado: dados_processados (ID: 73e2a80ce490442eb0682b2bcbd5ff00, Projeto: Datasets)


Uploading dataset changes (1 files compressed to 49.62 KiB) to https://files.clear.ml
File compression and upload completed: total size 49.62 KiB, 1 chunk(s) stored (average size 49.62 KiB)
ClearML Monitor: Could not detect iteration reporting, falling back to iterations as seconds-from-start
ClearML Monitor: Reporting detected, reverting back to iteration based reporting


2026-02-10 12:37:22,800 - INFO - ✓ Dataset criado (ID: 73e2a80ce490442eb0682b2bcbd5ff00)
2026-02-10 12:37:22,800 - INFO - 
[5] Fechando task...
2026-02-10 12:37:33,320 - INFO - ✓ Task finalizada e pronta para próximo pipeline
2026-02-10 12:37:33,320 - INFO - 
2026-02-10 12:37:33,322 - INFO - PIPELINE CONCLUÍDO COM SUCESSO
2026-02-10 12:37:33,322 - INFO - ================================================================================


In [4]:
dataset_anterior_id = resultado['dataset_id']

In [ ]:
# ============================================================================
# PIPELINE 2: ENGENHARIA DE FEATURES COM CLEARML
# ============================================================================
# Recebe o resultado do Pipeline 1 e aplica transformações de features
# ============================================================================

import traceback
from src.clearml.pipelines_clearml import executar_pipeline_features_clearml




resultado_features = executar_pipeline_features_clearml(
    df_processado=df_features,
    dataset_processado_id=dataset_anterior_id,
    offline_mode=False,
    # Configurações de features
    aplicar_codificacao=True,
    metodo_codificacao='label',
    criar_features_derivadas=True,
    tipos_features_derivadas=['imc', 'imc_classe', 'heat_index', 'dew_point', 't*u'],
    aplicar_normalizacao=True,
    metodo_normalizacao='standard'
)


2026-02-10 12:37:33,344 - INFO - Modo ONLINE: Iniciando integração ClearML
2026-02-10 12:37:33,345 - INFO - ✓ Credenciais ClearML carregadas do .env
2026-02-10 12:37:33,347 - INFO -   API Host: https://api.clear.ml
2026-02-10 12:37:33,348 - INFO -   Web Host: https://app.clear.ml
2026-02-10 12:37:33,349 - INFO - ✓ ClearML configurado para uso ONLINE
2026-02-10 12:38:42,603 - WARNING - Retrying (Retry(total=237, connect=240, read=237, redirect=240, status=240)) after connection broken by 'RemoteDisconnected('Remote end closed connection without response')': /v2.23/tasks.get_by_id
2026-02-10 12:39:09,902 - WARNING - Retrying (Retry(total=236, connect=240, read=236, redirect=240, status=240)) after connection broken by 'RemoteDisconnected('Remote end closed connection without response')': /v2.23/tasks.get_by_id
2026-02-10 12:39:45,194 - WARNING - Retrying (Retry(total=235, connect=240, read=235, redirect=240, status=240)) after connection broken by 'RemoteDisconnected('Remote end closed c

ClearML Task: created new task id=be0540b7be854c5783724ea7b04a13d8
ClearML results page: https://app.clear.ml/projects/06661800c2e642339d895b6d02c06481/experiments/be0540b7be854c5783724ea7b04a13d8/output/log


2026-02-10 12:45:31,656 - WARNING - Retrying (Retry(total=237, connect=240, read=237, redirect=240, status=240)) after connection broken by 'RemoteDisconnected('Remote end closed connection without response')': /v2.23/tasks.get_by_id
2026-02-10 12:45:33,647 - WARNING - Retrying (Retry(total=237, connect=240, read=237, redirect=240, status=240)) after connection broken by 'RemoteDisconnected('Remote end closed connection without response')': /v2.23/events.add_batch
2026-02-10 12:45:55,414 - WARNING - Retrying (Retry(total=237, connect=240, read=237, redirect=240, status=240)) after connection broken by 'RemoteDisconnected('Remote end closed connection without response')': /v2.23/tasks.get_all
2026-02-10 12:45:58,934 - WARNING - Retrying (Retry(total=236, connect=240, read=236, redirect=240, status=240)) after connection broken by 'RemoteDisconnected('Remote end closed connection without response')': /v2.23/tasks.get_by_id
2026-02-10 12:46:22,733 - WARNING - Retrying (Retry(total=236, co

🎨 Iniciando pipeline de FEATURES...
  1️⃣ Criando features derivadas (5 tipos)...
  2️⃣ Aplicando codificação (label)...
  3️⃣ Aplicando normalização (standard)...
✅ Pipeline FEATURES concluído! Shape final: (1720, 78)
   Novas colunas criadas: 37


2026-02-10 12:50:12,419 - INFO - DataFrame registrado: dados_features (shape: (1720, 78))
2026-02-10 12:50:21,083 - INFO - Arquivo registrado: artefatos_features


PermissionError: [WinError 32] O arquivo já está sendo usado por outro processo: 'temp_artefatos_features.json'

ClearML Monitor: Could not detect iteration reporting, falling back to iterations as seconds-from-start


2026-02-10 12:53:55,096 - WARNING - Retrying (Retry(total=237, connect=240, read=237, redirect=240, status=240)) after connection broken by 'RemoteDisconnected('Remote end closed connection without response')': /v2.23/tasks.get_all
2026-02-10 12:54:22,065 - WARNING - Retrying (Retry(total=237, connect=240, read=237, redirect=240, status=240)) after connection broken by 'RemoteDisconnected('Remote end closed connection without response')': /v2.23/events.add_batch
2026-02-10 12:54:22,398 - WARNING - Retrying (Retry(total=236, connect=240, read=236, redirect=240, status=240)) after connection broken by 'RemoteDisconnected('Remote end closed connection without response')': /v2.23/tasks.get_all
2026-02-10 12:54:49,370 - WARNING - Retrying (Retry(total=236, connect=240, read=236, redirect=240, status=240)) after connection broken by 'RemoteDisconnected('Remote end closed connection without response')': /v2.23/events.add_batch
2026-02-10 12:54:57,707 - WARNING - Retrying (Retry(total=235, con

In [ ]:
# ============================================================================
# PIPELINE 3: TREINAMENTO DE MODELOS COM CLEARML
# ============================================================================
# Recebe o resultado do Pipeline 2 e treina modelos com PyCaret
# ============================================================================

import traceback
from src.clearml.pipelines_clearml import executar_pipeline_treinamento_clearml

print(f"  Dataset ID anterior: {resultado_features.get('dataset_id', 'N/A')}")
df_para_treino = resultado_features['dados_features']
dataset_anterior_id = resultado_features.get('dataset_id')




resultado_treinamento = executar_pipeline_treinamento_clearml(
    df_features=df_para_treino,
    coluna_alvo='p1',
    tipo_problema='regressao',
    dataset_features_id=dataset_anterior_id,
    offline_mode=False,
    # Configurações de treinamento
    n_modelos_comparar=3,
    otimizar_hiperparametros=True,
    n_iter_otimizacao=10,
    salvar_modelo_final=True,
    nome_modelo="modelo_conforto_termico",
    pasta_modelos="../modelos"
)
    
# Melhor modelo
melhor_modelo_nome = str(resultado_treinamento['tabela_comparacao'].index[0])
print(f"\n🏆 Melhor Modelo: {melhor_modelo_nome}")
        
# Métricas
print(f"\n📊 Métricas do Melhor Modelo:")
metricas = resultado_treinamento['metricas_melhor']
metricas_principais = ['MAE', 'MSE', 'RMSE', 'R2', 'MAPE']
for nome in metricas_principais:
    if nome in metricas:
        valor = metricas[nome]
        if isinstance(valor, (int, float)):
            print(f"  • {nome}: {valor:.4f}")
        
        # Model ID
        model_id = resultado_treinamento.get('model_id', 'N/A')
        if model_id and model_id != 'N/A':
            model_id_display = model_id
        else:
            model_id_display = "N/A (modo offline ou erro)"
        
        print(f"\n💾 Modelo:")
        print(f"  • Salvo em: {resultado_treinamento.get('caminho_modelo', 'N/A')}")
        print(f"  • ClearML ID: {model_id_display}")
        
        # Comparação de modelos
        print(f"\n📋 Comparação de Modelos (Top {len(resultado_treinamento['tabela_comparacao'])}):")
        print(resultado_treinamento['tabela_comparacao'][['MAE', 'MSE', 'RMSE', 'R2']].head())
        
        print(f"\n🌐 Acesse: https://app.clear.ml")
        print(f"  • Projeto: Estudo de Sensação Térmica Humana em Santa Maria - RS")
        print(f"  • Task: Pipeline_Treinamento")
        print(f"  • Modelo: modelo_{melhor_modelo_nome}_modelo_conforto_termico")
        print(f"  • Genealogia: dados_brutos → processados → features → **MODELO**")
        
